In [1]:
# Imports
import numpy as np

In [2]:
# Import the tools
from sys import path
path.append('/Users/reid/dev/PythonCode/tm/tools')
path.append('D:/PythonCode/tm/tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin, e2, hardlim

In [3]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [4, 7]

In [4]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [5]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [6]:
# Network shape
layer_sizes

array([4, 7, 3])

In [7]:
# Inputs
inputs

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [8]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0]])

In [9]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [10]:
# Define a function to reset all the parameters
# This is done to test multiple different network types
def reset_params():
    weights_list.clear()
    biases_list.clear()
    n_list.clear()
    a_list.clear()
    s_list.clear()
    for i in range(1, len(layer_sizes)):
        weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
        biases_list.append(np.random.rand(layer_sizes[i], 1))
        n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        s_list.append(np.empty((layer_sizes[i], 1), dtype=float))

In [11]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [12]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [13]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])

    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [14]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

def train_network(learning_rate=.1, epochs=1):
    for i in range(epochs):
        # Iterate through the inputs and outputs
        for i in range(len(inputs)):
            # Run the network
            run_network(inputs[i].reshape(layer_sizes[0], 1))

            # Get the dialation matrix for the last layer
            F_dot = fill_F_dot(-1)

            # Calculate the sensitivites for the last layer
            s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

            # Update the weights and biases for the last layer
            weights_list[-1] += -learning_rate*np.matmul(s_list[-1], a_list[-2].T)
            biases_list[-1] += -learning_rate*s_list[-1]

            # Iterate through the remaining layers backwards
            for i in range(len(n_list)-2, 0, -1):
                s_list[i] = (1/layer_sizes[i+1]) * np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
                weights_list[i] += -learning_rate*np.matmul(s_list[i], a_list[i-1].T)
                biases_list[i] += -learning_rate*s_list[i]

In [15]:
# Function to evaluate the performance
# of the network
def performance():
    results = np.empty((len(inputs), layer_sizes[-1]), dtype=float)
    for i in range(len(inputs)):
        run_network(inputs[i].reshape(layer_sizes[0], 1))
        results[i] = outputs[i] - a_list[-1].flatten()
    return e2(results)

In [17]:
# Print performance
def print_performance_dict(performance_dict):
    for a in sorted(performance_dict.items(), key=lambda x: x[1]):
        print(f'Error: {a[1]} | ' + a[0])

In [18]:
# Trains netowrk using the parameters for each bit size
def update_performance_dict(layer_sizes_list, performance_dict):
    for layer_sizes in layer_sizes_list:
        # Get the inputs and outputs for the specifiq network size
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)
    
        # Iterate through the different learning rates
        # and epochs
        for rate in learning_rates:
            for epochs in epochs_list:
                # Reset weights and biases
                reset_params()
                # Train the network
                train_network(learning_rate=rate, epochs=epochs)
                # Update the performance dict
                performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': performance()})

In [18]:
### !!! Do not run code underneath until saving to the file
### It trains 22000 different networks
# Performance dictionnaries of the network
performance_dict_2bits = {}
performance_dict_4bits = {}
performance_dict_8bits = {}

In [19]:
# Parameters that won't change despite differnt layer sizes
# Set different learning rates
learning_rates = [1/(16*(2**i)) for i in range(10)]

# Set the diffrent epochs
epochs_list = [i for i in range(1, 21)]

In [20]:
# Multiple different layer sizes
# Start with 1 hidden layer
layer_sizes_list_2bits = [[4, 2*(2**i)]for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i)]for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i)]for i in range(10)]

In [21]:
# Calculate performance for 1 hidden layer
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [22]:
# 2 Hidden layers
# Redefine the transfer functions
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

# New layer sizes
layer_sizes_list_2bits = [[4, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]

In [23]:
# Calculate the performance for 2 hidden layers
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [24]:
print('-'*7 + '2bits' + '-'*7)
print_performance_dict(performance_dict_2bits)
print('-'*7 + '4bits' + '-'*7)
print_performance_dict(performance_dict_4bits)
print('-'*7 + '8bits' + '-'*7)
print_performance_dict(performance_dict_8bits)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [25]:
# Save the data of the performance dicts
performance_dicts = [performance_dict_2bits, performance_dict_4bits, performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'/Users/reid/dev/PythonCode/tm/BackPropagationBinaryCalcs/data/{2**(i+1)}bit_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])
### You can run code below this

In [73]:
# Chose a few networks from the data file to run tests on
# General parameters
learning_rates = [0.001953125, 0.00390625, 0.0078125]
epochs_list = [5, 14, 19]

In [74]:
# Start with one hidden layer
transfer_functions = [logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 2],
    [4, 32],
    [4, 64],
]
layer_sizes_list_4bits = [
    [8, 64],
    [8, 2],
    [8, 1024],
]
layer_sizes_list_8bits = [
    [16, 16],
    [16, 64],
    [16, 4],
]

In [75]:
# Write a function to calculate the average performance
def update_average_performance_dict(layer_sizes_list, average_performance_dict, n_samples):
    cumulated_performance = 0
    for layer_sizes in layer_sizes_list:
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)
        
        for rate in learning_rates:
                for epochs in epochs_list:
                    for i in range(n_samples):
                        reset_params()
                        train_network(learning_rate=rate, epochs=epochs)
                        cumulated_performance += performance()
                        
                    average_performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': cumulated_performance/n_samples})
                    cumulated_performance = 0

In [76]:
average_performance_dict_2bits = {}
average_performance_dict_4bits = {}
average_performance_dict_8bits = {}

In [77]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [78]:
# Two hidden layers
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 8, 2],
    [4, 2, 128],
    [4, 256, 32],
]
layer_sizes_list_4bits = [
    [8, 32, 64],
    [8, 1024, 64],
    [8, 2, 256],
]
layer_sizes_list_8bits = [
    [16, 8, 8],
    [16, 2, 1024],
    [16, 16, 4],
]

In [79]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [80]:
print('----------2 bit average----------')
print_performance_dict(average_performance_dict_2bits)
print('----------4 bit average----------')
print_performance_dict(average_performance_dict_4bits)
print('----------8 bit average----------')
print_performance_dict(average_performance_dict_8bits)

----------2 bit average----------
Error: 11.438259048327764 | Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.439852987994302 | Layers: [4, 64, 3], Learning rate: 0.0078125, Epochs: 19
Error: 11.442361063591502 | Layers: [4, 64, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.448469846065574 | Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19
Error: 11.453687797505607 | Layers: [4, 32, 3], Learning rate: 0.0078125, Epochs: 19
Error: 11.46746438596071 | Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 14
Error: 11.467831277122947 | Layers: [4, 64, 3], Learning rate: 0.0078125, Epochs: 14
Error: 11.472675909257687 | Layers: [4, 32, 3], Learning rate: 0.0078125, Epochs: 14
Error: 11.47586655242761 | Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.484570937473897 | Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 14
Error: 11.492762643650366 | Layers: [4, 64, 3], Learning rate: 0.00390625, Epochs: 14
Error: 11.496060584687758 | Laye

In [81]:
# Save the average performance data
performance_dicts = [average_performance_dict_2bits, average_performance_dict_4bits, average_performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'./data/{2**(i+1)}bit_average_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])

In [2]:
# Perform ceteris paribus test for 2 bist
# Read the average performance:
with open(f'./data/2bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [16]:
# Chose for which networks to test
test_layers = [
    [4, 32, 3],
    [4, 256, 32, 3],
    [4, 2, 3],
]

test_learning_rates = [
    0.00390625,
    0.001953125,
    0.0078125,
]

test_epochs = [
    19,
    14,
    5,
]

def check_error_variation_over_paramater(const1, const2, name1, name2, name_test):
    print(f'----------Error variation across {name_test}----------')
    for i in range(len(const1)):
        print(f'Network {i+1}')
        for performance in average_performance:
            if f'{name1}: {const1[i]}' in performance and f'{name2}: {const2[i]}' in performance:
                print(performance)

check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 5', 13.96723969350817)
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 14', 11.496060584687758)
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 19', 11.438259048327764)
Network 2
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 5', 15.881284908332372)
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 14', 14.292881999375885)
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 19', 13.680439863167805)
Network 3
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 5', 11.699962929062028)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 14', 11.46746438596071)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19', 11.448469846065574)
----------Error variation across learning rates----------
Network 1
('Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 19', 11.669080706327712)
('Layers: [4, 32, 3], Le

In [17]:
# Perform the same test for 4 bits
# Read the average performance:
with open(f'./data/4bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [18]:
# Define the parameters
test_layers = [
    [8, 2, 5],
    [8, 64, 5],
    [8, 32, 64, 5],
]

test_learning_rates = [
    0.0078125,
    0.001953125,
    0.00390625,
]

test_epochs = [
    19,
    14,
    5,
]

# Check error variation over epochs, learning rate and layers
check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 5', 11.704525991789666)
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 14', 11.462021206073375)
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 19', 11.433945720047964)
Network 2
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 5', 21.746346232443155)
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 14', 12.290378378945464)
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 19', 11.673666686532453)
Network 3
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 5', 14.90347309946306)
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 14', 12.857085247778597)
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 19', 12.318327628507607)
----------Error variation across learning rates----------
Network 1
('Layers: [8, 2, 5], Learning rate: 0.001953125, Epochs: 19', 11.67550311674572)
('Layers: [8, 2, 5], Learnin

In [21]:
# Perform the same test for 8 bits
# Read the average performance:
with open(f'./data/8bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [22]:
# Define the parameters
test_layers = [
    [16, 4, 9],
    [16, 64, 9],
    [16, 2, 1024, 9],
]

test_learning_rates = [
    0.0078125,
    0.001953125,
    0.00390625,
]

test_epochs = [
    19,
    14,
    5,
]

# Check error variation over epochs, learning rate and layers
check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 5', 11.685835594479505)
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 14', 11.479390423462124)
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 19', 11.424976080084498)
Network 2
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 5', 21.704807836707953)
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 14', 12.307892859410407)
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 19', 11.685280313203044)
Network 3
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 5', 14.928288258772605)
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 14', 12.825856862625956)
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 19', 12.323777815955141)
----------Error variation across learning rates----------
Network 1
('Layers: [16, 4, 9], Learning rate: 0.001953125, Epochs: 19', 11.670402302770329)
('Layers: [16

In [6]:
# Come back and test all the 2 bit network networks
# over the learning rates
# Define parameters
test_layers = [
    [4, 2, 3],
    [4, 32, 3],
    [4, 64, 3],
    [4, 8, 2, 3],
    [4, 2, 128, 3],
    [4, 256, 32, 3],
]

test_epochs = [
    5, 
    14, 
    19
]

In [15]:
# Order network performance for all the corresponding networks
for i, layers in enumerate(test_layers):
    for j, epochs in enumerate(test_epochs):
        print(f'Network {i*len(test_epochs) + j + 1}\nLayers: {layers} | Epochs: {epochs}')
        count += 1
        for performance in average_performance:
                    if f'Layers: {layers}' in performance and f'Epochs: {epochs}' in performance:
                        print(performance)

Network 1
Layers: [4, 2, 3] | Epochs: 5
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 5', 21.49681846289604)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 5', 13.938296407927314)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 5', 11.699962929062028)
Network 2
Layers: [4, 2, 3] | Epochs: 14
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 14', 12.26949627279391)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 14', 11.484570937473897)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 14', 11.46746438596071)
Network 3
Layers: [4, 2, 3] | Epochs: 19
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 19', 11.68712032655317)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 19', 11.47586655242761)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19', 11.448469846065574)
Network 4
Layers: [4, 32, 3] | Epochs: 5
('Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 5', 22.04548981749302)
('Layers: [4, 32, 3], Learning rate

In [133]:
# TODO: test stable network for whole range of epochs and record results
# TODO: test extremely simplified network structures see beneath as well
# TODO: test an extremely complexe network structure (just a lot of layers)
# TODO: test stable network performance on hard lim transfer functions
# TODO: test stable network performance but training it with a hardlim transfer function last layer
# TODO: redo all this with symetrical network

In [77]:
# Testing two network structures trained with huge numbers of epochs
# Network 1:
# Define layers
layer_sizes = [4, 32]

# Get inputs and outputs
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

# Append last layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

# Define transfer functions
transfer_functions = [logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [79]:
# Test for epochs going from 100 to 5000
for i in range(1, 51):
    # Reset all network parameters
    reset_params()
    
    print(f'-----Epochs: {i*100}-----')
    
    # Train the network
    train_network(learning_rate=0.00390625, epochs=i*100)

    # Get performance
    print(performance())

-----Epochs: 100-----
11.473762959309722
-----Epochs: 200-----
10.532396406522869
-----Epochs: 300-----
10.12144204062843
-----Epochs: 400-----
10.290725748428743
-----Epochs: 500-----
10.104415746563296
-----Epochs: 600-----
9.309831364143424
-----Epochs: 700-----
9.764292849680155
-----Epochs: 800-----
9.38949387499126
-----Epochs: 900-----
8.94903054696219
-----Epochs: 1000-----
9.226459386467514
-----Epochs: 1100-----
9.058519106846475
-----Epochs: 1200-----
9.32118015975078
-----Epochs: 1300-----
9.162462348419588
-----Epochs: 1400-----
9.113487149803225
-----Epochs: 1500-----
8.778421413974739
-----Epochs: 1600-----
8.837510256295028
-----Epochs: 1700-----
9.034953626484196
-----Epochs: 1800-----
9.014154445549504
-----Epochs: 1900-----
8.714761581070508
-----Epochs: 2000-----
8.73954371335011
-----Epochs: 2100-----
8.934335997599733
-----Epochs: 2200-----
8.953927938700605
-----Epochs: 2300-----
8.538826558788962
-----Epochs: 2400-----
8.714385182387483
-----Epochs: 2500-----
8.

In [41]:
# Netowrk 2:
# Layer sizes
layer_sizes = [4, 256, 32]

# Inputs and outputs
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

# Append S
layer_sizes = np.array(layer_sizes + [S], dtype=int)

# Define transfer functions
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

In [30]:
# Test for epochs going from 100 to 5000
for i in range(1, 51):
    # Reset all network parameters
    reset_params()
    
    print(f'-----Epochs: {i*100}-----')
    
    # Train the network
    train_network(learning_rate=0.0078125, epochs=i*100)

    # Get performance
    print(performance())

-----Epochs: 100-----
37.236467515819584
-----Epochs: 200-----
35.786332583191516
-----Epochs: 300-----
34.13565569717387
-----Epochs: 400-----
40.34397612097744
-----Epochs: 500-----
36.00178298953087
-----Epochs: 600-----
31.32184158323833
-----Epochs: 700-----
35.6275961581573
-----Epochs: 800-----


KeyboardInterrupt: 

In [42]:
print(layer_sizes)
for i in range(100):
    reset_params()
    train_network(learning_rate=0.0078125, epochs=14)
    print(performance())

[  4 256  32   3]
17.183216448290473
17.183216448290466
17.18321644829047
17.183216448290484
17.183216448290498
17.183216448290477
17.183216448290473
17.183216448290487
17.18321644829047
17.183216448290473
17.18321644829047
17.183216448290477
17.183216448290466
17.183216448290477
17.18321644829047
17.183216448290477
17.18321644829048
17.183216448290473
17.18321644829049
17.183216448290494
17.183216448290466
17.18321644829049
17.18321644829048
17.18321644829049
17.183216448290477
17.183216448290473
17.18321644829047
17.183216448290494
17.183216448290477
17.183216448290484
17.183216448290473
17.183216448290473
17.18321644829049
17.18321644829047
17.183216448290466
17.183216448290477
17.183216448290462
17.183216448290473
17.18321644829047
17.183216448290477
17.183216448290473
17.18321644829048
17.18321644829047
17.18321644829049
17.183216448290494
17.183216448290473
17.183216448290462
17.183216448290473
17.183216448290484
17.183216448290473
17.18321644829049
17.183216448290487
17.18321644

In [38]:
print(layer_sizes)
cum_sum = 0
for i in range(100):
    reset_params()
    train_network(learning_rate=0.0078125, epochs=14)
    cum_sum += performance()
print(cum_sum/100)

[  4   2 128   3]
38.46221565424534


In [70]:
transfer_functions = [logsig, hardlim]

In [74]:
performance()

26.0

In [72]:
run_network(inputs[0].reshape(len(inputs[0]), 1))

In [73]:
print(a_list[-1])
print(outputs[3])

[[1]
 [1]
 [1]]
[0 1 1]


In [13]:
# Test the model with keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [14]:
model = Sequential()
model.add(Dense(layer_sizes[1], input_shape=(layer_sizes[0],), activation='sigmoid'))
model.add(Dense(layer_sizes[2], activation='linear'))

In [15]:
sdg = SGD(.9)
model.compile(loss="categorical_crossentropy", optimizer=sdg,
	metrics=["accuracy"])
model.fit(inputs, outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.5000 - loss: 7.0425


In [16]:
model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[ 0.01186371, -8.282264  , -1.4656483 ],
       [ 0.0887537 , -9.469966  , -1.5549538 ],
       [ 0.15937938, -8.323184  , -1.3819622 ],
       [ 0.22767666, -9.4916    , -1.4718292 ],
       [-0.01628557, -8.611647  , -1.6199257 ],
       [ 0.04203045, -9.72632   , -1.6907108 ],
       [ 0.14307892, -8.662352  , -1.5352875 ],
       [ 0.20412186, -9.765932  , -1.603532  ]], dtype=float32)